In [1]:
!pip install faster-whisper gradio torchaudio soundfile resampy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.4 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import numpy as np
import torch
import time
import soundfile as sf
import resampy
import subprocess
from faster_whisper import WhisperModel

device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float16" if device == "cuda" else "float32"
model_size = "large-v2"
model = WhisperModel(model_size, device=device, compute_type=compute_type)

def vad_transcribe(audio_file, threshold = 0.5, min_speech_ms = 250, max_speech_s = 20, min_silence_ms = 200):
    try:
        if audio_file is None:
            return "[Error] No file provided.", None, None, None, None

        audio_data, sr = sf.read(audio_file)
        if len(audio_data.shape) > 1:
            audio_data = np.mean(audio_data, axis=1)
        audio_16k = resampy.resample(audio_data, sr, 16000).astype(np.float32)

        total_duration = len(audio_16k) / 16000.0

        start_time = time.time()

        segments, _ = model.transcribe(
            audio_16k,
            vad_filter=True,
            vad_parameters={
                "threshold": threshold,
                "min_speech_duration_ms": min_speech_ms,
                "max_speech_duration_s": max_speech_s,
                "min_silence_duration_ms": min_silence_ms
            }
        )

        end_time = time.time()
        latency = round(end_time - start_time, 2)

        transcript = ""
        speech_duration = 0
        segment_count = 0

        for segment in segments:
            transcript += f"[{segment.start:.2f}s - {segment.end:.2f}s]: {segment.text.strip()}\n"
            speech_duration += (segment.end - segment.start)
            segment_count += 1

        speech_ratio = round((speech_duration / total_duration) * 100, 2) if total_duration else 0

        if device == "cuda":
            subprocess.run(["nvidia-smi"])

        return (
            transcript.strip(),
            f"{latency} seconds"
        )

    except Exception as e:
        return f"[Error] {str(e)}", None

with gr.Blocks() as demo:
    gr.Markdown("## 🎤 Voice Activity Detection + Transcription using Faster-Whisper + Gradio")
    gr.Markdown("Upload an audio file and apply configurable Voice Activity Detection.")

    with gr.Row():
        with gr.Column():
            audio_input = gr.Audio(label="Upload Audio File", type="filepath")
            transcribe_btn = gr.Button("Transcribe with VAD")

    with gr.Row():
        transcript_output = gr.Textbox(label="📋 Transcription with Timestamps", lines=12)

    with gr.Row():
        latency_output = gr.Textbox(label="⏱️ Latency")

    transcribe_btn.click(
        fn=vad_transcribe,
        inputs=[audio_input],
        outputs=[transcript_output, latency_output]
    )

demo.launch()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

vocabulary.txt: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7683747339404ec3dc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
